<a href="https://colab.research.google.com/github/GundetiManoj99/pdfquerys/blob/main/pdfquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q cassio datasets langchain openai tiktoken

In [9]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00


In [10]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio

In [11]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.4 MB/s eta 0:00:00


In [12]:
from PyPDF2 import PdfReader

In [27]:
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:jeZOqGyohwmlkCXMtcwhUFZh:f41e4d0fcb8bcffab9af1df1d2839c459ccbebf84f693ba3473e12f146775ae8"
ASTRA_DB_ID = "bbd69670-7e35-4b31-bdb8-55896259e8b2"

OPENAI_API_KEY = "sk-proj-***************************************"

In [21]:
!pip install --upgrade astrapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.2 MB/s eta 0:00:00


In [15]:
pdfreader = PdfReader('test.pdf')

In [16]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [17]:
raw_text

'Generating FlavorMoleculesUsingScientificMachineLearning\nLuanaP.Queiroz,CarineM.Rebello,ErbetA.Costa,ViníciusV.Santana,BrunoC.L.Rodrigues,\nAlírioE.Rodrigues, AnaM.Ribeiro,andIdelfonso B.R.Nogueira*\nCiteThis:ACSOmega2023,8,10875−10887\n ReadOnline\nACCESS\n Metrics &More\n Article Recommendations +sıSupporting Information\nABSTRACT: Flavorisanessentialcomponent inthedevelop-\nmentofnumerous products inthemarket.Theincreasing\nconsumption ofprocessed andfastfoodandhealthypackaged\nfoodhasupraised theinvestment innewflavoring agentsand\nconsequently inmolecules withflavoring properties. Inthis\ncontext, thisworkbringsupascientific machine learning\n(SciML) approach toaddressthisproductengineering need.\nSciMLincomputational chemistry hasopenedpathsinthe\ncompound’s propertyprediction withoutrequiring synthesis. This\nworkproposesanovelframework ofdeepgenerative modelswithin\nthiscontexttodesignnewflavormolecules. Through theanalysis\nandstudyofthemolecules obtained fromthegenerative m

In [28]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [29]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [30]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)

In [31]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [32]:
texts[:50]

['Generating FlavorMoleculesUsingScientificMachineLearning\nLuanaP.Queiroz,CarineM.Rebello,ErbetA.Costa,ViníciusV.Santana,BrunoC.L.Rodrigues,\nAlírioE.Rodrigues, AnaM.Ribeiro,andIdelfonso B.R.Nogueira*\nCiteThis:ACSOmega2023,8,10875−10887\n ReadOnline\nACCESS\n Metrics &More\n Article Recommendations +sıSupporting Information\nABSTRACT: Flavorisanessentialcomponent inthedevelop-\nmentofnumerous products inthemarket.Theincreasing\nconsumption ofprocessed andfastfoodandhealthypackaged\nfoodhasupraised theinvestment innewflavoring agentsand\nconsequently inmolecules withflavoring properties. Inthis\ncontext, thisworkbringsupascientific machine learning\n(SciML) approach toaddressthisproductengineering need.\nSciMLincomputational chemistry hasopenedpathsinthe',
 'context, thisworkbringsupascientific machine learning\n(SciML) approach toaddressthisproductengineering need.\nSciMLincomputational chemistry hasopenedpathsinthe\ncompound’s propertyprediction withoutrequiring synthesis. This\nwor

In [33]:

astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 50 headlines.


In [34]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your question (or type 'quit' to exit): what neural networks are used for synthesis of flavor molecules?

QUESTION: "what neural networks are used for synthesis of flavor molecules?"


ANSWER: "The neural networks used for synthesis of flavor molecules in this context are deep generative models."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9363] "replicated byothersynthetic molecules. Therecreation and
analysisofthesechemicals al ..."
    [0.9311] "ongenerative neuralnetworkmodels.
2.METHODOLOGY ANDRESULTS
2.1.Database. Thedatabase ..."
    [0.9304] "context, thisworkbringsupascientific machine learning
(SciML) approach toaddressthis ..."
    [0.9291] "Generating FlavorMoleculesUsingScientificMachineLearning
LuanaP.Queiroz,CarineM.Rebe ..."

What's your next question (or type 'quit' to exit): what are GGNN and what is its use

QUESTION: "what are GGNN and what is its use"


ANSWER: "GGNN stands for Gated Graph Neural Network. It is a type of neural network used for processing graph-structured data, where the data is represented as a graph consisting of nodes and edges. GGNN is used for tasks such as generating and deconstructing molecules."

FIRST DOCUMENTS BY RELEVANCE:


    [0.8674] "Then,thestartingnodeisselected,andthesequential nodes’
orderisdefinedaccording tothe ..."
    [0.8581] "REINVENT, AAE,VAE,ORGAN, LatentGAN andGraph-
INVENT. Forthisstudy,theauthorstraineda ..."
    [0.8580] "feedforward artificialneuralnetwork.Theglobalreadoutblock
Figure2.Co-occurrence heat ..."
    [0.8563] "W e r isatrainable
weighttensorinrregarding theedgelabel,
e;bisalearnable
parameter; ..."

What's your next question (or type 'quit' to exit): quit
